In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/soccer/database.sqlite


# About Dataset
## [European Soccer Database](https://www.kaggle.com/hugomathien/soccer):
* 25k+ matches, players & teams attributes for European Professional Football
* +25,000 matches
* +10,000 players
* 11 European Countries with their lead championship
* Seasons 2008 to 2016
* Players and Teams' attributes* sourced from EA Sports' FIFA video game series, including the weekly updates
* Team line up with squad formation (X, Y coordinates)
* Betting odds from up to 10 providers
* Detailed match events (goal types, possession, corner, cross, fouls, cards etc…) for +10,000 matches

In [2]:
## Importing Libraries
import numpy as np
import pandas as pd
import sqlite3 as sql
import matplotlib.pyplot as plt

In [3]:
#path = "../input/soccer"  #Our data path is input/soccer/database.sqlite
database = "../input/soccer/database.sqlite"

# Connecting with database to analyze data

In [4]:
# Create a SQL connection to our SQLite database
conn = sql.connect(database)

# reading tables with pandas
tables = pd.read_sql("Select * FROM sqlite_master WHERE type = 'table';", con=conn)

# print(tables)
tables

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


![](http://)Now we can see the `name`, `tbl_name`, and `sql query` used to create

### Exploring Tables

In [5]:
Teams = pd.read_sql("SELECT * FROM Team;", conn)
Teams

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB
...,...,...,...,...,...
294,49479,10190,898.0,FC St. Gallen,GAL
295,49837,10191,1715.0,FC Thun,THU
296,50201,9777,324.0,Servette FC,SER
297,50204,7730,1862.0,FC Lausanne-Sports,LAU


We have 299 teams

In [6]:
# Listing Countries
Desh = pd.read_sql("select * from Country;", conn)
Desh

,id,name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy
5,13274,Netherlands
6,15722,Poland
7,17642,Portugal
8,19694,Scotland
9,21518,Spain


We have 11 Countries in Total

### Listing Leagues

In [7]:
Leagues = pd.read_sql("SELECT * FROM League;", conn)
Leagues

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A
5,13274,13274,Netherlands Eredivisie
6,15722,15722,Poland Ekstraklasa
7,17642,17642,Portugal Liga ZON Sagres
8,19694,19694,Scotland Premier League
9,21518,21518,Spain LIGA BBVA


Now how to find the leagues in respective country:
    For eg Some Popular Leagues are
*     Spain -> LALIGA
*     England -> Premier League
* German -> Bundesliga

Now to find these infromation we can apply join between countries and leagues
* Here 2 tables are `Desh` and `Leagues`
* Both have common columns Desh.id and Leagues.country_id

In [8]:
# viewing above creared variables
Leagues, Desh

(       id  country_id                      name
 0       1           1    Belgium Jupiler League
 1    1729        1729    England Premier League
 2    4769        4769            France Ligue 1
 3    7809        7809     Germany 1. Bundesliga
 4   10257       10257             Italy Serie A
 5   13274       13274    Netherlands Eredivisie
 6   15722       15722        Poland Ekstraklasa
 7   17642       17642  Portugal Liga ZON Sagres
 8   19694       19694   Scotland Premier League
 9   21518       21518           Spain LIGA BBVA
 10  24558       24558  Switzerland Super League,
        id         name
 0       1      Belgium
 1    1729      England
 2    4769       France
 3    7809      Germany
 4   10257        Italy
 5   13274  Netherlands
 6   15722       Poland
 7   17642     Portugal
 8   19694     Scotland
 9   21518        Spain
 10  24558  Switzerland)

In [9]:
type(Leagues), type(Desh), type(database), type(tables)

(pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 str,
 pandas.core.frame.DataFrame)

In [10]:
# Teams = pd.read_sql("""SELECT Desh.name, Leagues.name FROM Desh JOIN 
#                     Leagues ON
#                     Desh.id = Leagues.country_id;""", conn)
# Teams

In [12]:
leagues = pd.read_sql("""SELECT Country.name, League.name
                        FROM League
                        JOIN Country ON Country.id = League.country_id;""", conn)
leagues

,name,name
0,Belgium,Belgium Jupiler League
1,England,England Premier League
2,France,France Ligue 1
3,Germany,Germany 1. Bundesliga
4,Italy,Italy Serie A
5,Netherlands,Netherlands Eredivisie
6,Poland,Poland Ekstraklasa
7,Portugal,Portugal Liga ZON Sagres
8,Scotland,Scotland Premier League
9,Spain,Spain LIGA BBVA


### List of Matches